# Tabular Data
We're deviating a little from the previous structure of the course because we can't really talk about ggplot without having some basic mastery of manipulating tabular data.  
  
## Discreteness, Continuity, Smoothness, Mathematical Structure, Philosophy
![](./mathematics.png)
**Question: Which is primary, numbers or the universe?**  
Two takes:  
1. Numbers pre-exist the universe, or the exist independently of the universe, and we know things about these real things called numbers which have an independent reality. 
2. Numbers are observations about regularities in the structure of the universe.  
  
## Relational Data
Relational data is structured information organized in tables (or relations) with rows representing individual records and columns representing data attributes or properties.  
Lucky for us, data is always finite and countable and that means even
things like continuous functions (eg, voltage traces) are represented by a finite number of samples and that means (in principle) every data set is a table.  
  
![](./chet_dogfood.png)

In [8]:
suppressPackageStartupMessages(library(tidyverse))
frat_wide <- suppressWarnings(read_csv(
    "source_data/frat_boys_basic.csv",
    show_col_types = FALSE
))
head(frat_wide,3)

name,fraternity,address,major_name,major_address,enrollment_date,gpa,hometown,hs_gpa,intended_minor,sat_math,act_composite,languages,club_interest,dietary_preference
<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Patrick McPatrick,Alpha Rho Psi,"12 Franklin St, Chapel Hill, NC",Philosophy of Sandwiches,101 Hoagie Hall,2021-08-15,2.9,Wilmington NC,3.3,Linguistics,640,NA,NA,"debate_team,philosophy_club",vegetarian
Chet Dogfood,Omega Delta Nu,"42 Rosemary Ln, Chapel Hill, NC",Comparative Surf Studies,12 Wave Science Center,2020-08-20,3.1,Savannah GA,NA,NA,NA,NA,NA,"surfing,beer_pong",NA
Brock Sturdybench,Sigma Tau Phi,"88 Cameron Ave, Chapel Hill, NC",Applied Meteorology,300 Skywatch Tower,2022-01-10,2.7,Madison WI,NA,NA,NA,29,NA,storm_chasing,NA


Consider the above table. Notice it's a pretty wide dataset.  
  
It is the kind of thing we might like to get if we are a data scientist. Spmetimes, we won't get something this wide. If someone asks you to check out a *database* for them you might instead get all the data above broken up into multiple tables.  
  
See below for an example of that...

In [3]:
frat_student <- suppressWarnings(
  read_csv("source_data/student.csv", show_col_types = FALSE)
)
head(frat_student,3)

id,name,fraternity_id,major_id,address_id,enrollment_date,gpa
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>
1,Patrick McPatrick,1,1,1,2021-08-15,2.9
2,Chet Dogfood,2,2,2,2020-08-20,3.1
3,Brock Sturdybench,3,3,3,2022-01-10,2.7


In [4]:
frat_address <- suppressWarnings(
  read_csv("source_data/address.csv", show_col_types = FALSE)
)
head(frat_address,3)

id,line1,city,state
<dbl>,<chr>,<chr>,<chr>
1,12 Franklin St,Chapel Hill,NC
2,42 Rosemary Ln,Chapel Hill,NC
3,88 Cameron Ave,Chapel Hill,NC


In [5]:
frat_fraternity <- suppressWarnings(
  read_csv("source_data/fraternity.csv", show_col_types = FALSE)
)
head(frat_fraternity,3)

id,name,address_id
<dbl>,<chr>,<dbl>
1,Alpha Rho Psi,28
2,Omega Delta Nu,29
3,Sigma Tau Phi,30


In [6]:
frat_major <- suppressWarnings(
  read_csv("source_data/major.csv", show_col_types = FALSE)
)
head(frat_major,3)

id,name,address_id
<dbl>,<chr>,<dbl>
1,Philosophy of Sandwiches,21
2,Comparative Surf Studies,22
3,Applied Meteorology,23


Notice all of these datasets are linked by id. Enter... **joins and pivots!**  
  
## Pivots
If we first understand that we want one **observation per row** then pivots
make sense.  
Consider our original table:  

In [7]:
head(frat_wide,3)

name,fraternity,address,major_name,major_address,enrollment_date,gpa,hometown,hs_gpa,intended_minor,sat_math,act_composite,languages,club_interest,dietary_preference
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Patrick McPatrick,Alpha Rho Psi,"12 Franklin St, Chapel Hill, NC",Philosophy of Sandwiches,101 Hoagie Hall,2021-08-15,2.9,Wilmington NC,3.3,Linguistics,640,NA,NA,"debate_team,philosophy_club",vegetarian
Chet Dogfood,Omega Delta Nu,"42 Rosemary Ln, Chapel Hill, NC",Comparative Surf Studies,12 Wave Science Center,2020-08-20,3.1,Savannah GA,NA,NA,NA,NA,NA,"surfing,beer_pong",NA
Brock Sturdybench,Sigma Tau Phi,"88 Cameron Ave, Chapel Hill, NC",Applied Meteorology,300 Skywatch Tower,2022-01-10,2.7,Madison WI,NA,NA,NA,29,NA,storm_chasing,NA


**Two ways to think of observations:**
1. Wide data: one person per row (shown above)
2. Long data: a person and a recording of some data  
  
From the second point of view, one row has multiple observations. Depending on how we are defining an observation, we may want to *pivot* to a different conception of an observation.  

### Pivot Longer

In [32]:
frat_long <- pivot_longer(frat_wide, fraternity:dietary_preference,
    names_to = "property",
    values_to = "observation",
    values_transform = function(x) ifelse(is.na(x), NA, as.character(x)))

head(frat_long, 14)

name,property,observation
<chr>,<chr>,<chr>
Patrick McPatrick,fraternity,Alpha Rho Psi
Patrick McPatrick,address,"12 Franklin St, Chapel Hill, NC"
Patrick McPatrick,major_name,Philosophy of Sandwiches
Patrick McPatrick,major_address,101 Hoagie Hall
Patrick McPatrick,enrollment_date,2021-08-15
Patrick McPatrick,gpa,2.9
Patrick McPatrick,hometown,Wilmington NC
Patrick McPatrick,hs_gpa,3.3
Patrick McPatrick,intended_minor,Linguistics


**Important Note**: In order to do this, all of our columns need to be character variables. This is why, when we included `values_transform`,  `values_transform = function(x) ifelse(is.na(x), NA, as.character(x)))`, which first checks if a value is missing/NA, and converts non-missing values to character.  

**Ways to deal with this issue:**
1. **What we did** - convert everything to characters and reconvert later.
2. Pivot multiple times based on the types of columns we have and then join the data back together, resulting in a frame where each row is a value observation but with multiple columns, one per type.
3. Use a list column and wrap our elements so we can have a heterogeneous
column.

**How much data do we have on each student?**  
Using the `problems()` function, we see in row 12 of `frat_boys_basic.csv`, read_csv() found only 14 columns, but the header has 15 columns.  
By printing row 12, we see that all 15 columns exist, but the csv row ended without a trailing comma, so it can look like one fewer column, even though the value of the final column is `NA`.

In [28]:
problems(frat_wide)
frat_wide[12, ]

row,col,expected,actual,file
<int>,<int>,<chr>,<chr>,<chr>
12,14,15 columns,14 columns,/home/jovyan/lecture5/source_data/frat_boys_basic.csv


name,fraternity,address,major_name,major_address,enrollment_date,gpa,hometown,hs_gpa,intended_minor,sat_math,act_composite,languages,club_interest,dietary_preference
<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Freddy Picklejuice,Omega Delta Nu,"200 E Franklin St, Chapel Hill",Philosophy of Sandwiches,101 Hoagie Hall,2021-01-11,2.8,Richmond VA,3.1,NA,NA,NA,"English,French",chess_club,NA


Then, we can get the number of observations we have for each student by making a table of frat_long, where we tally the number of observations by name.

In [29]:
frat_long %>% 
    group_by(name) %>%  
    tally() %>% 
    arrange(desc(n))

name,n
<chr>,<int>
Brock Sturdybench,14
Bruce Jamboree,14
Chadwick Boomboom,14
Chet Dogfood,14
Colin Thunderhug,14
Dirk Hambone,14
Duncan McChortle,14
Finn Pickleton,14
Freddy Picklejuice,14


### Pivot Wider
We can also undo what we did above. 

In [34]:
wide_data <- frat_long %>%
  pivot_wider(
    id_cols = name,
    names_from = property,
    values_from = observation
  )

head(wide_data,3)

name,fraternity,address,major_name,major_address,enrollment_date,gpa,hometown,hs_gpa,intended_minor,sat_math,act_composite,languages,club_interest,dietary_preference
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Patrick McPatrick,Alpha Rho Psi,"12 Franklin St, Chapel Hill, NC",Philosophy of Sandwiches,101 Hoagie Hall,2021-08-15,2.9,Wilmington NC,3.3,Linguistics,640,NA,NA,"debate_team,philosophy_club",vegetarian
Chet Dogfood,Omega Delta Nu,"42 Rosemary Ln, Chapel Hill, NC",Comparative Surf Studies,12 Wave Science Center,2020-08-20,3.1,Savannah GA,NA,NA,NA,NA,NA,"surfing,beer_pong",NA
Brock Sturdybench,Sigma Tau Phi,"88 Cameron Ave, Chapel Hill, NC",Applied Meteorology,300 Skywatch Tower,2022-01-10,2.7,Madison WI,NA,NA,NA,29,NA,storm_chasing,NA


## Joins
They are called joins because they join data from multiple tables. Because data is often intentionally separated into different data sets in databases, or related but from entirely different sources or data sets, we need to do joins.  
  
### Definitions:
1. **table**: a data structure with some columns (usually named) and some rows.
2. **observation**: one item of interest (often one row, but not always)
3. **key**: a column or columns (almost always unique) which identifies an observation
4. **join: the process of combining tables (almost always using keys)
5. **query**: a program which uses some tables, joins, filters, subsets columns, creates new columns, and returns a new table
6. **relational data(base)**: a collection of tables organized along the above lines
7. **pivot**: when we change our notion of observation we "pivot" our data so that we have one row per observation.
  
### Left Join
First, we want to *combine students with their majors* and we want to know which students may not yet have chosen a major. This is called a **"left" join** because we keep everything in the left table and fill in NA's if we don't have data in the right table. 

In [9]:
students <- suppressWarnings(read_csv("source_data/student.csv", show_col_types = FALSE))
majors <- suppressWarnings(read_csv("source_data/major.csv", show_col_types = FALSE))
fraternities <- suppressWarnings(read_csv("source_data/fraternity.csv", show_col_types = FALSE))
addresses <- suppressWarnings(read_csv("source_data/address.csv", show_col_types = FALSE))
props <- suppressWarnings(read_csv("source_data/student_properties.csv", show_col_types = FALSE))

# LEFT JOIN: keep all students, attach majors when present
print(paste(
  "students:", nrow(students),
  "majors:", nrow(majors)
))
students_majors_left <- students %>%
  left_join(majors %>% rename(major_name = name, major_address_id = address_id),
            by = c("major_id" = "id")) %>%
  select(id, name, major_id, major_name, enrollment_date, gpa)

print(paste("left join rows:", nrow(students_majors_left)))
students_majors_left

[1] "students: 20 majors: 9"
[1] "left join rows: 20"


id,name,major_id,major_name,enrollment_date,gpa
<dbl>,<chr>,<dbl>,<chr>,<date>,<dbl>
1,Patrick McPatrick,1,Philosophy of Sandwiches,2021-08-15,2.9
2,Chet Dogfood,2,Comparative Surf Studies,2020-08-20,3.1
3,Brock Sturdybench,3,Applied Meteorology,2022-01-10,2.7
4,Dirk Hambone,4,Forestry and Folklore,2021-08-22,3.2
5,Lance Beefcastle,5,Environmental Design,2020-08-18,3.4
6,Troy Thunderpants,6,Digital History,2022-08-19,2.8
7,Chadwick Boomboom,NA,NA,2021-08-17,3.0
8,Miles Cheesewheel,4,Forestry and Folklore,2020-08-24,3.3
9,Finn Pickleton,5,Environmental Design,2022-01-12,2.6


### Inner Join
Next, let's show an **inner join** with a tiny data set.
![](joins.png)

In [14]:
# CONTRIVED INNER JOIN: only rows with matching keys survive
left_df <- tibble(id = c(1, 2, 3), val_left = c("A", "B", "C"))
right_df <- tibble(id = c(2, 3, 4), val_right = c("X", "Y", "Z"))

print("left")
left_df

print("right")
right_df

print(paste(
  "left rows:", nrow(left_df),
  "right rows:", nrow(right_df)
))

inner_df <- left_df %>% inner_join(right_df, by = "id")

print(paste("inner rows:", nrow(inner_df)))
print("inner join")
inner_df

[1] "left"


id,val_left
<dbl>,<chr>
1,A
2,B
3,C


[1] "right"


id,val_right
<dbl>,<chr>
2,X
3,Y
4,Z


[1] "left rows: 3 right rows: 3"
[1] "inner rows: 2"
[1] "inner join"


id,val_left,val_right
<dbl>,<chr>,<chr>
2,B,X
3,C,Y


### Right Join and Full Join
Now, let's show a **right join** and a **full join**.

In [15]:
print(paste(
  "students:", nrow(students),
  "properties:", nrow(props)
))
props_right <- students %>%
  right_join(props, by = c("id" = "student_id"))

# Row count after right join
print(paste("right join rows:", nrow(props_right)))

# Observe the intentionally dangling properties with student_id = 99
props_right %>% filter(is.na(name))

# FULL JOIN: keep all students and all properties
props_full <- students %>%
  full_join(props, by = c("id" = "student_id"))

print(paste("full join rows:", nrow(props_full)))
props_full

[1] "students: 20 properties: 62"
[1] "right join rows: 62"


id,name,fraternity_id,major_id,address_id,enrollment_date,gpa,id.y,property,value
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<chr>,<chr>
99,NA,NA,NA,NA,NA,NA,61,hometown,Nowhereville ZZ
99,NA,NA,NA,NA,NA,NA,62,hs_gpa,3.9


[1] "full join rows: 62"


id,name,fraternity_id,major_id,address_id,enrollment_date,gpa,id.y,property,value
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<chr>,<chr>
1,Patrick McPatrick,1,1,1,2021-08-15,2.9,1,hometown,Wilmington NC
1,Patrick McPatrick,1,1,1,2021-08-15,2.9,2,hs_gpa,3.3
1,Patrick McPatrick,1,1,1,2021-08-15,2.9,3,intended_minor,Linguistics
2,Chet Dogfood,2,2,2,2020-08-20,3.1,4,hometown,Savannah GA
2,Chet Dogfood,2,2,2,2020-08-20,3.1,5,sat_math,690
2,Chet Dogfood,2,2,2,2020-08-20,3.1,6,intended_minor,Statistics
3,Brock Sturdybench,3,3,3,2022-01-10,2.7,7,hometown,Madison WI
3,Brock Sturdybench,3,3,3,2022-01-10,2.7,8,hs_gpa,3.1
3,Brock Sturdybench,3,3,3,2022-01-10,2.7,9,club_interest,storm_chasing


### Semi Joins and Anti Joins
What about **semi joins** and **anti joins**?

In [16]:
# SEMI JOIN: which students have at least one property row?
print(paste("students:", nrow(students)))
has_props <- students %>% semi_join(props, by = c("id" = "student_id"))

# ANTI JOIN: which students have no properties? (may be empty)
no_props <- students %>% anti_join(props, by = c("id" = "student_id"))

print(paste(
  "with props:", nrow(has_props),
  "without props:", nrow(no_props)
))
has_props
no_props

[1] "students: 20"
[1] "with props: 20 without props: 0"


id,name,fraternity_id,major_id,address_id,enrollment_date,gpa
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>
1,Patrick McPatrick,1,1,1,2021-08-15,2.9
2,Chet Dogfood,2,2,2,2020-08-20,3.1
3,Brock Sturdybench,3,3,3,2022-01-10,2.7
4,Dirk Hambone,4,4,4,2021-08-22,3.2
5,Lance Beefcastle,5,5,5,2020-08-18,3.4
6,Troy Thunderpants,1,6,6,2022-08-19,2.8
7,Chadwick Boomboom,2,NA,7,2021-08-17,3.0
8,Miles Cheesewheel,3,4,8,2020-08-24,3.3
9,Finn Pickleton,4,5,9,2022-01-12,2.6


id,name,fraternity_id,major_id,address_id,enrollment_date,gpa
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>


What about joining look ups?

In [17]:
# Join fraternities to their address; note missing address for one fraternity
print(paste(
  "fraternities:", nrow(fraternities),
  "addresses:", nrow(addresses)
))
frats_with_addresses <- fraternities %>%
  left_join(addresses %>% rename(
    frat_street = line1, frat_city = city, frat_state = state
  ), by = c("address_id" = "id"))

print(paste("frat left join rows:", nrow(frats_with_addresses)))
frats_with_addresses

# Join students to their own address; note a missing student address_id
print(paste(
  "students:", nrow(students),
  "addresses:", nrow(addresses)
))
students_with_address <- students %>%
  left_join(addresses %>% rename(
    student_street = line1, student_city = city, student_state = state
  ), by = c("address_id" = "id"))

print(paste("student left join rows:", nrow(students_with_address)))
students_with_address

[1] "fraternities: 5 addresses: 32"
[1] "frat left join rows: 5"


id,name,address_id,frat_street,frat_city,frat_state
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,Alpha Rho Psi,28,10 Alpha House,Chapel Hill,NC
2,Omega Delta Nu,29,20 Omega House,Chapel Hill,NC
3,Sigma Tau Phi,30,30 Sigma House,Chapel Hill,NC
4,Beta Gamma Mu,31,40 Beta Gamma House,Chapel Hill,NC
5,Rho Kappa Chi,NA,NA,NA,NA


[1] "students: 20 addresses: 32"
[1] "student left join rows: 20"


id,name,fraternity_id,major_id,address_id,enrollment_date,gpa,student_street,student_city,student_state
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<chr>,<chr>,<chr>
1,Patrick McPatrick,1,1,1,2021-08-15,2.9,12 Franklin St,Chapel Hill,NC
2,Chet Dogfood,2,2,2,2020-08-20,3.1,42 Rosemary Ln,Chapel Hill,NC
3,Brock Sturdybench,3,3,3,2022-01-10,2.7,88 Cameron Ave,Chapel Hill,NC
4,Dirk Hambone,4,4,4,2021-08-22,3.2,21 McCorkle Pl,Chapel Hill,NC
5,Lance Beefcastle,5,5,5,2020-08-18,3.4,77 Columbia St,Chapel Hill,NC
6,Troy Thunderpants,1,6,6,2022-08-19,2.8,9 Franklin Ct,Chapel Hill,NC
7,Chadwick Boomboom,2,NA,7,2021-08-17,3.0,300 Hillsborough St,Chapel Hill,NC
8,Miles Cheesewheel,3,4,8,2020-08-24,3.3,150 Henderson St,Chapel Hill,NC
9,Finn Pickleton,4,5,9,2022-01-12,2.6,12 Pittsboro St,Chapel Hill,NC


What about chaining joins?

In [19]:
# Build a richer table by chaining left joins to keep all students
print(paste("students:", nrow(students)))
student_profile <- students %>%
  left_join(majors %>% rename(major_name = name, major_address_id = address_id),
            by = c("major_id" = "id")) %>%
  left_join(fraternities %>% rename(frat_name = name, frat_address_id = address_id),
            by = c("fraternity_id" = "id")) %>%
  left_join(addresses %>% rename(
    student_street = line1, student_city = city, student_state = state
  ), by = c("address_id" = "id")) %>%
  left_join(addresses %>% rename(
    major_street = line1, major_city = city, major_state = state
  ), by = c("major_address_id" = "id")) %>%
  left_join(addresses %>% rename(
    frat_street = line1, frat_city = city, frat_state = state
  ), by = c("frat_address_id" = "id")) %>%
  select(
    id, name, major_name, frat_name,
    student_street, student_city,
    major_street, major_city,
    frat_street, frat_city
  )

print(paste("profile rows:", nrow(student_profile)))
student_profile

[1] "students: 20"
[1] "profile rows: 20"


id,name,major_name,frat_name,student_street,student_city,major_street,major_city,frat_street,frat_city
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Patrick McPatrick,Philosophy of Sandwiches,Alpha Rho Psi,12 Franklin St,Chapel Hill,101 Hoagie Hall,Chapel Hill,10 Alpha House,Chapel Hill
2,Chet Dogfood,Comparative Surf Studies,Omega Delta Nu,42 Rosemary Ln,Chapel Hill,12 Wave Science Center,Chapel Hill,20 Omega House,Chapel Hill
3,Brock Sturdybench,Applied Meteorology,Sigma Tau Phi,88 Cameron Ave,Chapel Hill,300 Skywatch Tower,Chapel Hill,30 Sigma House,Chapel Hill
4,Dirk Hambone,Forestry and Folklore,Beta Gamma Mu,21 McCorkle Pl,Chapel Hill,NA,NA,40 Beta Gamma House,Chapel Hill
5,Lance Beefcastle,Environmental Design,Rho Kappa Chi,77 Columbia St,Chapel Hill,42 EcoDome,Chapel Hill,NA,NA
6,Troy Thunderpants,Digital History,Alpha Rho Psi,9 Franklin Ct,Chapel Hill,12 Time Machine Annex,Chapel Hill,10 Alpha House,Chapel Hill
7,Chadwick Boomboom,NA,Omega Delta Nu,300 Hillsborough St,Chapel Hill,NA,NA,20 Omega House,Chapel Hill
8,Miles Cheesewheel,Forestry and Folklore,Sigma Tau Phi,150 Henderson St,Chapel Hill,NA,NA,30 Sigma House,Chapel Hill
9,Finn Pickleton,Environmental Design,Beta Gamma Mu,12 Pittsboro St,Chapel Hill,42 EcoDome,Chapel Hill,40 Beta Gamma House,Chapel Hill


### Summary of Joins
1. **left join** – keeps all rows from the left table and adds matching rows from the right table, filling with `NA` when there’s no match.
2. **inner join** – keeps only the rows that have matching keys in both tables.
3. **right join** – keeps all rows from the right table and adds matching rows from the left table, filling with `NA` when there’s no match.
4. **full join** – keeps all rows from both tables, filling with `NA` when there’s no match on either side.
5. **semi join** – returns only the rows from the left table that have a match in the right table, without adding columns from the right table.
6. **anti join** – returns only the rows from the left table that do not have a match in the right table.
7. **joining lookups** – using a table of reference values to add descriptive information or labels to another table based on a key.
8. **chaining joins** – performing multiple joins sequentially to combine more than two tables step by step.